In [14]:
import pandas as pd
import os
from IPython.display import HTML

best_brawlers = 12

df = pd.read_csv('maps/best_brawlers.csv')

def clean_brawler_name(brawler):
    if brawler=='8-Bit':
        return brawler
    elif brawler=='Mr.P':
        return 'Mr. P'
    else:
        return brawler.replace('-',' ')
    
def clean_string(s):
    s = s.split(', ')
    s = [(x.replace('%','')) for x in s]
    return ', '.join(s)

def round_string(s):
    s = s.split(', ')
    s = [str(int(float(x))) for x in s]
    return ', '.join(s)
    
def nth_repl(s, sub, repl, n):
    find = s.find(sub)
    # If find is not -1 we have found at least one match for the substring
    i = find != -1
    # loop util we find the nth or we find no match
    while find != -1 and i != n:
        # find + 1 means we start searching from after the last match
        find = s.find(sub, find + 1)
        i += 1
    # If i is equal to n we found nth match so replace
    if i == n:
        return s[:find] + repl + s[find+len(sub):]
    return s

images_dir = './maps/brawler_images'
images_path = os.listdir(images_dir)
gamemodes_dir = './maps/gamemode_images'

images = {}
for image in images_path:
    images[clean_brawler_name(image.replace('.png',''))] = os.path.join(images_dir, image)
    
list_of_gamemodes = df['gamemodes']

for gamemode in list_of_gamemodes:
    fname = gamemode.replace(' ','-')+'.png'
    images[gamemode] = os.path.join(gamemodes_dir, fname)
    
columns = [str(x) for x in list(range(1,best_brawlers+1))]
    
df[columns] = df['best_brawlers'].str.split(', ', best_brawlers-1, expand=True)

def path_to_image_html(path):
    return '<img src="'+ path + '" width="80" >'
    
def path_to_image_html_smaller(path):
    return '<img src="'+ path + '" width="40" >'

format_dict = {}
for image_col in columns+['gamemodes']:
    if image_col=='gamemodes':
        format_dict[image_col] = path_to_image_html_smaller
    else:
        format_dict[image_col] = path_to_image_html
        
for col in columns+['gamemodes']:
    df[col] = df[col].map(images)

df = df.drop(['best_brawlers'],axis=1)
df['win_rates'] = df['win_rates'].map(clean_string)
df['win_rates'] = df['win_rates'].map(round_string)
df['win_rates'] = df['win_rates'].map(lambda s: nth_repl(s, ", ","<br>",best_brawlers/3))
df['use_rates'] = df['use_rates'].map(clean_string)
df['use_rates'] = df['use_rates'].map(lambda s: nth_repl(s, ", ","<br>",best_brawlers/3))
df = df[['gamemodes','map']+columns+['win_rates','use_rates']]
df.index += 1
# df.to_html('best_brawlers.html', escape=False, formatters=format_dict, render_links=True)

In [15]:
df2 = df.fillna('   ').head(9)
HTML(df2.to_html(escape=False, formatters=format_dict))

,gamemodes,map,1,2,3,4,5,6,7,8,9,10,11,12,win_rates,use_rates
1,,Shooting Star,,,,,,,,,,,,,"60, 60, 60, 5955, 55, 55, 54, 52, 52, 52, 51","9.2, 3.4, 2.2, 2.518.3, 2.8, 5.0, 10.4, 7.2, 2.6, 3.6, 3.0"
2,,Canal Grande,,,,,,,,,,,,,"67, 65, 63, 6359, 58, 57, 55, 54, 54, 54, 54","2.3, 2.1, 2.1, 2.43.0, 2.1, 2.1, 2.3, 2.6, 10.4, 6.3, 2.1"
3,,Snake Prairie,,,,,,,,,,,,,"71, 63, 59, 5756, 54, 52, 52, 52, 49, 49, 45","6.9, 2.3, 8.8, 9.33.7, 2.3, 19.9, 2.3, 2.6, 2.3, 2.8, 12.0"
4,,Super Beach,,,,,,,,,,,,,"58, 58, 57, 5655, 55, 54, 54, 54, 54, 53, 53","2.3, 2.3, 2.8, 3.23.3, 2.3, 2.1, 4.8, 4.2, 2.9, 2.1, 2.3"
5,,Pinhole Punt,,,,,,,,,,,,,"57, 57, 56, 5655, 55, 54, 54, 54, 53, 53, 52","3.4, 4.4, 2.9, 2.12.9, 2.5, 2.6, 2.2, 3.5, 2.5, 3.2, 2.3"
6,,Sneaky Fields,,,,,,,,,,,,,"60, 59, 58, 5756, 55, 54, 54, 53, 53, 53, 52","3.3, 2.9, 2.7, 2.94.1, 4.2, 2.3, 2.1, 3.0, 3.8, 3.7, 4.5"
7,,Hard Rock Mine,,,,,,,,,,,,,"57, 56, 56, 5555, 55, 53, 52, 52, 52, 52, 52","2.9, 5.6, 3.3, 2.92.4, 9.4, 2.3, 3.0, 2.6, 4.2, 2.3, 7.8"
8,,Chill Space,,,,,,,,,,,,,"81, 78, 76, 7473, 72, 71, 70, 70, 69, 69, 69","5.0, 3.2, 3.1, 2.74.7, 3.3, 2.5, 2.4, 2.5, 5.0, 4.2, 6.7"
9,,Double Swoosh,,,,,,,,,,,,,"63, 58, 57, 5655, 55, 53, 53, 51, 51, 50, 50","2.2, 3.4, 4.2, 5.68.6, 5.8, 5.1, 2.9, 2.5, 3.3, 6.6, 2.7"


In [16]:
df3 = df.fillna('   ').tail(9)
HTML(df3.to_html(escape=False, formatters=format_dict))

,gamemodes,map,1,2,3,4,5,6,7,8,9,10,11,12,win_rates,use_rates
10,,Hot Potato,,,,,,,,,,,,,"75, 60, 58, 5858, 57, 55, 55, 54, 54, 54, 53","9.8, 7.6, 6.8, 11.43.7, 3.0, 5.3, 13.6, 3.7, 2.3, 6.1, 3.0"
11,,Safe Zone,,,,,,,,,,,,,"65, 60, 59, 5757, 56, 54, 53, 52, 51, 51, 50","4.6, 7.7, 4.1, 2.13.7, 12.4, 3.0, 3.2, 9.7, 4.8, 8.4, 2.5"
12,,Cant touch this,,,,,,,,,,,,,"62, 62, 60, 5958, 58, 58, 57, 57, 55, 54, 54","2.7, 2.2, 7.4, 11.12.8, 19.4, 2.8, 2.8, 2.8, 2.8, 3.5, 2.1"
13,,Ring of Fire,,,,,,,,,,,,,"62, 60, 60, 5958, 58, 57, 57, 56, 54, 54, 52","3.3, 4.0, 2.5, 7.52.5, 2.3, 2.5, 3.3, 2.5, 11.7, 10.8, 3.3"
14,,Dueling Beetles,,,,,,,,,,,,,"100, 90, 81, 6462, 61, 60, 59, 57, 56, 55, 55","7.4, 10.3, 5.4, 2.52.7, 2.9, 2.9, 2.9, 10.0, 2.9, 3.4, 4.4"
15,,Open Zone,,,,,,,,,,,,,"74, 72, 71, 6865, 64, 64, 61, 59, 58, 57, 57","3.3, 2.8, 3.3, 2.82.5, 9.4, 3.3, 5.6, 2.2, 2.2, 2.8, 2.2"
16,,Belles Rock,,,,,,,,,,,,,"68, 62, 60, 6059, 59, 59, 58, 58, 58, 57, 56","3.9, 2.4, 2.2, 3.32.5, 6.7, 5.8, 16.7, 2.6, 4.8, 3.8, 10.0"
17,,Goldarm Gulch,,,,,,,,,,,,,"57, 55, 54, 5353, 53, 52, 51, 51, 50, 50, 48","9.5, 5.6, 13.0, 11.83.7, 5.6, 10.3, 3.9, 2.1, 22.2, 2.6, 2.9"
18,,X Marks the Spot,,,,,,,,,,,,,"69, 65, 61, 5959, 57, 57, 55, 55, 55, 54, 54","8.3, 3.3, 3.6, 2.22.5, 5.0, 13.3, 3.3, 4.2, 5.0, 6.7, 3.3"
